In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Test generation
prompt = "Correct this sentence: 'he go to store yesterday'"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
output = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Correct this sentence: 'he go to store yesterday' 
The correct sentence is: 'he went to the store yesterday.'

The change was made to the verb 'go' to the past tense 'went' instead of the past participle 'go'.


In [3]:
import json
transcript_path_test = "/data/willett_data/model_transcriptions/neurips_transformer_time_masked_seed_0.json"
with open(transcript_path_test) as f:
    model_outputs = json.load(f)

In [4]:
from huggingface_hub import login
login()

In [5]:

def build_llama_1B(modelName="meta-llama/Llama-3.2-1B-Instruct",
                cacheDir=None,
                device="auto",
                load_in_8bit=False,
                auth_token=True,        # NEW: pass your HF token for the gated repo
                dtype="auto"):          # NEW: pick bfloat16/float16 automatically
   

    # 1 Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        modelName,
        cache_dir=cacheDir,
        token=auth_token          # required after you’ve accepted Meta’s license
    )

    # 2 Load model
    torch_dtype = {"auto": None,
                   "bf16": torch.bfloat16,
                   "fp16": torch.float16}.get(dtype, None)

    model = AutoModelForCausalLM.from_pretrained(
        modelName,
        cache_dir=cacheDir,
        device_map=device,        # "auto", "cpu", "cuda", or explicit dict
        load_in_8bit=load_in_8bit,
        token=auth_token,
        torch_dtype=torch_dtype   # saves VRAM if you pick bf16/fp16
    )

    # 3 Padding tweaks (Llama uses no official pad token)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    model.config.pad_token_id = tokenizer.pad_token_id

    return model, tokenizer

In [6]:
MODEL_CACHE_DIR = "/data/willett_data/llama"
llm, llm_tokenizer = build_llama_1B(
cacheDir=MODEL_CACHE_DIR, device="auto", load_in_8bit=True
)

OSError: Can't load tokenizer for 'meta-llama/Llama-3.2-1B-Instruct'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-3.2-1B-Instruct' is the correct path to a directory containing all relevant files for a LlamaTokenizerFast tokenizer.